# Main

In [18]:
import pandas as pd
# import random

In [19]:
ds = pd.read_csv("./perfect_datasets/perfect_dataset_v2.csv")

In [20]:
ds_overview = pd.read_csv("./perfect_datasets/perfect_dataset_v1_overview.csv")

In [28]:
similarity = pd.read_csv("./perfect_datasets/perfect_datasets_v4.csv")

In [42]:
temp = pd.DataFrame()
temp["title"] = similarity["name"].str.lower().str.split()
temp["title"] = similarity["name"].apply(lambda x: x.replace(":", " :"))

In [43]:
for_sugg = pd.DataFrame()
for_sugg["title"] = similarity["name"].str.lower().str.split()
for_sugg["title"] = similarity["name"].apply(lambda x: x.replace("", ""))

In [44]:

movie_sugg = []
c = 0

def recommend(movie):
    global c
    global movie_sugg
    if(ds[ds["title"].str.lower() == movie.lower()].empty):
        for i in temp["title"]:
            tmp = i.split(",")
            movi = tmp[0].lower().split()
            if(movie.lower() in movi) and c==0:
                movie_sugg.append(tmp[0].replace(" :", ":"))
            if(movie.lower() in movi) and c==1:
                movie_sugg = (tmp[0].replace(" :", ":"))
        print("did you mean: ", movie_sugg, end="\n\n") if c<=1 else print("No movie found")
        c += 1
        return
    elif(movie == ""):
        print("Enter a movie name")
        return
    
    for_sugg["title"] = for_sugg["title"].apply(lambda x: x.split(","))
    
    index = ds[(ds["title"].str.lower()) == movie.lower()].index[0]

    moviess = for_sugg["title"][index][1:]
    rndm = [1,2,3,4,5]
    
    '''
    for i in range(0,5):
        rd = random.randint(1,10)
        rndm.append(rd if rd not in rndm else random.randint(1,10))
    '''

    for i in rndm:
        index2 = ds[(ds["title"].str.lower()) == moviess[i].lower()].index[0]
        print("Movie Id:",ds.iloc[index2]["movie_id"])
        print("Movie Name:",moviess[i].replace(" :", ":"))
        print("Movie Overview:",ds_overview.iloc[index2]["overview"], end="\n\n")


In [45]:
inp = str(input("Enter the movie name: "))
recommend(inp)

if movie_sugg != []:
        movie_re_input = str(input("Enter the movie name: "))
        recommend(movie_re_input)


Movie Id: 122917
Movie Name: The Hobbit: The Battle of the Five Armies
Movie Overview: Immediately after the events of The Desolation of Smaug, Bilbo and the dwarves try to defend Erebor's mountain of treasure from others who claim it: the men of the ruined Laketown and the elves of Mirkwood. Meanwhile an army of Orcs led by Azog the Defiler is marching on Erebor, fueled by the rise of the dark lord Sauron. Dwarves, elves and men must unite, and the hope for Middle-Earth falls into Bilbo's hands.

Movie Id: 82702
Movie Name: How to Train Your Dragon 2
Movie Overview: The thrilling second chapter of the epic How To Train Your Dragon trilogy brings back the fantastical world of Hiccup and Toothless five years later. While Astrid, Snotlout and the rest of the gang are challenging each other to dragon races (the island's new favorite contact sport), the now inseparable pair journey through the skies, charting unmapped territories and exploring new worlds. When one of their adventures leads

## Chintan TfidfVectorizer

In [8]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from difflib import get_close_matches  # Python's difflib module for finding close matches

# Load data
ds = pd.read_csv("./perfect_datasets/perfect_dataset_v2.csv")

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(ds['title'])

# Calculate the cosine similarity between movies
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

def find_similar_movies(movie_name):
    similar_movies = []
    for title in ds['title']:
        similar_movies.extend(get_close_matches(movie_name, [title], n=5, cutoff=0.6))
    return list(set(similar_movies))

def recommend(movie_name):
    movie_name = movie_name.lower()
    
    # Check if the exact movie name exists
    exact_match = ds[ds['title'].str.lower() == movie_name]
    
    if not exact_match.empty:
        idx = exact_match.index[0]
        sim_scores = list(enumerate(cosine_sim[idx]))
        # sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:]  # Get the top 5 most similar movies
        movie_indices = [i[0] for i in sim_scores]
        recommended_movies = ds['title'].iloc[movie_indices]
        print("Recommended Movies:")
        print(recommended_movies)
    else:
        print("Movie not found. Did you mean:")
        similar_movies = find_similar_movies(movie_name)
        print(similar_movies)

# User input
inp = input("Enter a movie name: ")
recommend(inp)


Recommended Movies:
1       Pirates of the Caribbean: At World's End
2                                        Spectre
3                          The Dark Knight Rises
4                                    John Carter
5                                   Spider-Man 3
                          ...                   
4504                                      Cavite
4505                                   Newlyweds
4506                   Signed, Sealed, Delivered
4507                            Shanghai Calling
4508                           My Date with Drew
Name: title, Length: 4508, dtype: object
